In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, numpy.random as npr, pystan as ps, pickle, pandas
from pylab import plot, show, legend
from time import time
from scipy.stats import *
from tqdm import trange

In [2]:
sm = ps.StanModel(file="garch_t.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c2d2b11f53ac18648f7d8b74054e190a NOW.


In [6]:
dat = pandas.read_csv("la_pm_positive.csv").values
Y = dat[:,1]
T = len(Y)

In [7]:
p = 1
q = 1
r = np.maximum(p,q)
nu = 4

### Divide-and-conquer:

In [8]:
K = 10
tstarts = np.arange(K).astype(int)
tends = 1 + tstarts
tstarts *= int(T/K)
tends *= int(T/K)

In [9]:
n_chains = 10
for k in range(K) :
    start = time()
    tstart, tend = tstarts[k], tends[k]
    data = dict(T=tend-tstart, p=p, q=q, r=r, nu=nu, y=Y[tstart:tend], power=K)
    fit = sm.sampling(data=data, thin=1, n_jobs=min(10,n_chains), chains=n_chains, init="random", iter=1_000)
    file = open("/xtmp/DC-BATS_data/real_data/garch_t_chunk"+str(k+1)+"_mcmc.pkl", "wb")
    pickle.dump(fit.extract(), file)
    file.close()
    mle = sm.optimizing(data=data)
    file = open("/xtmp/DC-BATS_data/real_data/garch_t_chunk"+str(k+1)+"_mle.pkl", "wb")
    pickle.dump(fit.extract(), file)
    file.close()
    print(round((time()-start)/60,2), "minutes to run")

0.19 minutes to run
0.21 minutes to run
0.24 minutes to run


0.22 minutes to run
0.23 minutes to run
0.22 minutes to run
0.19 minutes to run


0.23 minutes to run
0.19 minutes to run
0.22 minutes to run


### MCMC on full data:

In [12]:
start = time()
data = dict(T=T, p=p, q=q, r=r, nu=nu, y=Y, power=1)
fit = sm.sampling(data=data, thin=1, n_jobs=min(10,n_chains), chains=n_chains, init="random", iter=1_000)
file = open("/xtmp/DC-BATS_data/real_data/garch_t_fulldata_mcmc.pkl", "wb")
pickle.dump(fit.extract(), file)
file.close()
print(round((time()-start)/60,2), "minutes to run")

3.32 minutes to run
